In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.linalg import solve
from utility import Newton_orbit, BrusselatorModel

import matplotlib.animation as animation


In [ ]:
def brusselator_inner(Dx, Dy,L,A,B,z_L,N):
    
    # Discretize space
    h = z_L / (N - 1)  # Grid spacing
    def Lap_mat(N):
        main_diag = -2  * np.ones(N)
        off_diag = np.ones(N - 1)
        laplacian = np.diag(main_diag) + np.diag(off_diag, k=1) + np.diag(off_diag, k=-1)
        return laplacian
    
    def dydt(t, y):
        X = y[:N-2]
        Y = y[N-2:] 

        # Central difference for ALL INNER POINTS (indices 1 to N-2)
        # Inner points (central difference)
       
        # Dirichlet BCs
        X_BCs = A*np.eye(1,N-2,0)[0] + A*np.eye(1,N-2,N-3)[0]
        Y_BCs = (B/A)*np.eye(1,N-2,0)[0] + (B/A)*np.eye(1,N-2,N-3)[0]
        
        d2Xdz2 = (1/h**2)*(Lap_mat(N-2)@X + X_BCs)
        d2Ydz2 = (1/h**2)* (Lap_mat(N-2)@Y + Y_BCs)
        # Reaction-diffusion equation
        dXdt = Dx/(L**2) * d2Xdz2 + Y*(X**2) - (B+1)*X + A
        dYdt = Dy/(L**2) * d2Ydz2 - Y*(X**2) + B*X

        dydt = np.concatenate([dXdt, dYdt])
        return dydt
    

    def brusselator_jacobian(t,y): # A faire : Stockage creuse de la matrice 
        X = y[:N-2]
        Y = y[N-2:]
        n = len(X) #The inner points of the mesh
        h = z_L / (N - 1)
        
        # Diffusion coefficients
        alpha_x = Dx / (L*h)**2
        alpha_y = Dy / (L*h)**2
        
        # Fill J_y_by_iter and J_YY (tridiagonal blocks)
        Jxx = alpha_x*Lap_mat(n) - (B+1)*np.eye(n) + 2*np.diag(X*Y)
        Jyy = alpha_y*Lap_mat(n) - np.diag(X**2)
        Jyx = np.diag(X**2)
        Jxy = B*np.eye(n) - 2*np.diag(X*Y)
        #Assembling the whole matrix
        top = np.hstack((Jxx, Jyx))  # Horizontal stacking of A11=M-I and A12=b
        bottom = np.hstack((Jxy,Jyy)) # Horizontal stacking of A21=c and A22=d
        J = np.vstack((top, bottom))  # Vertical stacking of the two rows
        return J
    #Phase conditions

    def s1(t,y): 
        
        return dydt(t,y)[0]
    
    def ds1_dy(t,y):
        return brusselator_jacobian(t,y)[0,:]
    def ds1_dT(t,y):

        return 0.0
    def s2(t,y,y_preced):
        return (y-y_preced)@dydt(t,y_preced)
    def ds2_dy(t,y):
        return dydt(t,y)
    return dydt, brusselator_jacobian, s1, ds1_dy, ds1_dT


## Trial Run

In [ ]:
# Example usage
if __name__ == "__main__":
    # Parameters
    Dx = 0.008
    Dy = 0.004
    L = 0.5130 #1.5 #1*0.5130 #characteristic length
    z_L = 1  #Domain length
    A,B = 2,5.45   
    N = 50 # Number of grid points
 
    
    # Initial condition (e.g., a Gaussian pulse)
    z = np.linspace(0, z_L, N)
    # perturb = 0.1 * np.exp(-((z - z_L / 2)**2)) #*np.concatenate([A*np.ones(N),(B/A)*np.ones(N)])
    perturb = np.sin(np.pi*(z/z_L))
    X0 = A + 0.1*perturb
    Y0 = B/A + 0.1*perturb
    # # Enforce Dirichlet BCs on the initial condition
    # Y0[0], Y0[-1] = B/A, B/A  
    # X0[0], X0[-1] = A, A
    y0 = np.concatenate([X0[1:-1],Y0[1:-1]])

    # Create the system
    # system = brusselator_1D(Dx, Dy, L, A,B,z_L,N)
    f, Jacf, s, grad_s,ds_T = brusselator_inner(Dx, Dy, L, A,B,z_L,N)
    tf = 200
    t_span = (0, tf)  # Time span
    sol = solve_ivp(f, t_span, y0, method='BDF', t_eval=np.linspace(0, tf, 1000), jac = Jacf)

In [ ]:
y_ini = sol.y[:,-1]
y_ini.shape

In [ ]:
# Plot the solution
plt.figure(figsize=(10, 6))
for i, t in enumerate(sol.t):
    if i % 900== 0:  # Plot every 50th time step
        plt.plot(z[1:-1], sol.y[:N-2, i],'+', label=f"X, t={t:.2f}")
        plt.plot(z[1:-1], sol.y[N-2:, i],'+', label=f"Y, t={t:.2f}")

plt.xlabel("z")
plt.ylabel("Concentrations X, Y")
plt.legend()
plt.title("Brusselator Model with Dirichlet BCs")
plt.show()

#Heat map 

In [ ]:
Xmean = sol.y[:N-2,:]
Xmean = np.mean(Xmean,axis = 0)
Ymean = sol.y[N-2:,:]
Ymean = np.mean(Ymean,axis = 0)

fig, ax = plt.subplots(1,2)
ax[0].plot(np.linspace(0, tf, 1000), Xmean, label = '<X>')
ax[0].plot(np.linspace(0, tf, 1000), Ymean, label='<Y>')
ax[0].legend()
ax[0].set_ylabel(f"$Concentrations$")
ax[0].set_xlabel("t")
ax[1].plot(Xmean,Ymean)
ax[1].set_ylabel(f"$<Y>$")
ax[1].set_xlabel(f"$<X>$")
fig.set_size_inches((10,5))
fig.suptitle(f'Brusselator Model with Dirichlet BCs:')
fig.subplots_adjust(left=0.09, bottom=0.1, right=0.95, top=0.90, hspace=0.35, wspace=0.55)
# plt.savefig(f'./Results/brusselator_1D.png')

## Newton orbite 

In [ ]:
if __name__ == "__main__":
    param_file = "./brusselator_params_4.in"  # JSON file containing model parameters
    model = BrusselatorModel(param_file)
    print("Loaded parameters:", model.L)

f = model.dydt
Jacf = model.brusselator_jacobian

In [ ]:
# Parameters
# Dx = 0.008
# Dy = 0.004
# L = 0.5130 #1.5 #1*0.5130 #characteristic length
# z_L = 1  
# A,B = 2,5.45   
# N = 50
# Initial condition (e.g., a Gaussian pulse)
z_L = model.z_L
N = model.N
A, B = model.A, model.B
z = np.linspace(0, z_L, N)
perturb = np.sin(np.pi*(z/z_L))

X0 = A + 0.01*perturb
Y0 = B/A + 0.01*perturb

y0 = np.concatenate([X0[1:-1],Y0[1:-1]])

# Create the system
# f, Jacf, s, grad_s,ds_T = brusselator_inner(Dx, Dy, L, A,B,z_L,N)


Max_iter = 100
epsilon = 1e-13
T_0 = model.T_ini

### Initial integration

Integrate sufficiently the equation to find a good starting value for the Newton method
We expect y_0 to be in the periodic orbit. 

In [ ]:
t_eval = np.linspace(0.0,20*T_0, 1000)

sol = solve_ivp(fun=f,t_span=[0.0, 20*T_0],
                t_eval=t_eval, 
                y0=y0, method='RK45', 
                **{"rtol": 1e-7,"atol":1e-9}
                )

y_T = sol.y[:,-1]
y_T.shape

In [ ]:
# f, Jacf, s, grad_s,ds_T = brusselator_inner(Dx, Dy, L, A,B,z_L,N)
k, T_by_iter, y_by_iter, Norm_B, Norm_Deltay, monodromy = Newton_orbit(f,y_T,T_0, Jacf,2, Max_iter, epsilon)



In [ ]:
eig, eigvec = np.linalg.eig(monodromy)
mask = np.abs(eig) - 1 >  1e-7
print("Number of Floquet multipliers outside the unit circle\n",len(eig[mask]))

print("Spectral radius of the Monodromy matrix:\n",np.max(np.abs(eig)))

In [ ]:
# Extract real and imaginary parts of eigenvalues
real_parts = np.real(eig)
imaginary_parts = np.imag(eig)
print(imaginary_parts.shape)
# Create the figure and axis
fig, ax = plt.subplots(figsize=(6, 6))

# Plot the unit circle
theta = np.linspace(0, 2 * np.pi, 1000)
circle_x = np.cos(theta)
circle_y = np.sin(theta)
ax.plot(circle_x, circle_y, 'k--', label='Unit Circle')

# Plot the eigenvalues
ax.scatter(real_parts, imaginary_parts, color='r', label='Eigenvalues')

# Set labels and title
ax.set_xlabel(f'Re($\lambda$)')
ax.set_ylabel(f'Im($\lambda$)')
ax.set_title(f'Eigenvalues of the Monodromy matrix on Complex Plane\n with L = {model.L}')

# Set equal aspect ratio
ax.set_aspect('equal', 'box')

# Add grid, legend, and plot
ax.grid(True)
# ax.legend()

# Show the plot
plt.show()

In [ ]:
print(y_by_iter[k-1][:N-2][23])
print(y_by_iter[k-1][N-2:][23])
print(T_by_iter[k-1])
k

In [ ]:
Tab = np.asarray(y_by_iter[:k-1])
X = Tab[:,N-2:]
Y = Tab[:,:N-2]


In [ ]:
fig, ax = plt.subplots(2,2,sharex='all')
ax[0,0].plot(np.arange(k-1),X.mean(axis=1),'+-')
ax[0,0].set_ylabel(f"$<X>$")

ax[0,1].plot(np.arange(k-1),Y.mean(axis=1),'+-')
ax[0,1].set_ylabel(f"$<Y>$")
ax[1,0].semilogy(np.arange(k),Norm_Deltay[:k],'x-')
ax[1,0].set_xlabel("Newton iterations")
ax[1,0].set_ylabel(f"$\parallel \Delta X \parallel$")
ax[1,1].semilogy(np.arange(k),Norm_B[:k],'x-')
ax[1,1].set_xlabel("Newton iterations")
ax[1,1].set_ylabel(f"$\parallel \phi(X^*(0),T) - X^*(T) \parallel$")
# ax[1,1].set_ylabel(f"$\parallel (r,s) \parallel_2$")
fig.set_size_inches((8,8))
fig.suptitle(f'Brusselator model: $L=%.4f$ \n $T = %.4f$ ' % (model.L, T_by_iter[k-1]))
fig.subplots_adjust(left=0.09, bottom=0.1, right=0.95, top=0.90, hspace=0.35, wspace=0.55)
# plt.savefig(f'./Results/Modulated_Laser_T_known_m_{str(m)}.png')

In [ ]:
y0 = np.array(y_by_iter[k-1])
t_eval = np.linspace(0.0,20*T_by_iter[k-1], 1000)

sol = solve_ivp(fun=f,t_span=[0.0, 20*T_by_iter[k-1]],
                t_eval=t_eval, 
                y0=y0, method='RK45', 
                **{"rtol": 1e-7,"atol":1e-9}
                )

y_T = sol.y[:,-1]
y_T.shape

In [ ]:
Xmean = sol.y[:N-2,:]
#Xmean = np.mean(Xmean,axis = 0)
Ymean = sol.y[N-2:,:]
#Ymean = np.mean(Ymean,axis = 0)

In [ ]:
fig, ax = plt.subplots(1,2)
ax[0].plot(t_eval, Xmean[23], label = '<X>')
ax[0].plot(t_eval, Ymean[23], label='<Y>')
ax[0].legend()
ax[0].set_ylabel(f"$Concentrations$")
ax[0].set_xlabel("t")
ax[1].plot(Xmean[23],Ymean[23])
ax[1].set_ylabel(f"$<Y>$")
ax[1].set_xlabel(f"$<X>$")
fig.set_size_inches((10,5))
fig.suptitle(f'Brusselator Model with Dirichlet BCs:')
fig.subplots_adjust(left=0.09, bottom=0.1, right=0.95, top=0.90, hspace=0.35, wspace=0.55)
# plt.savefig(f'./Results/brusselator_1D.png')